<a href="https://colab.research.google.com/github/shuvayanb/LFF-for-design-and-optimisation/blob/main/Newtonian_3D_body.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<font size="6">LFF: Rocket nose cone</font>

This Jupyter Notebook is a guiding demonstration of how simple algebraic equations based on local-inclination approaches can be cast into a low-fidelity framework (LFF) for computing pressure acting on a body in high-speed regime. Although approximate, these methods enable quick estimation of pressure forces acting on the body which is very cost-effective, especially for initial design phase. At the end of this notebook you will learn the following:

1. Read a unstructured surface mesh generated using [Gmsh](https://gmsh.info/)
2. Implement Newtonian theory on every mesh element to compute local pressure
3. Visualise the flowfield using [Paraview](https://www.paraview.org/)

In [ ]:
import numpy as np
import math
import itertools
from itertools import islice

![](https://raw.githubusercontent.com/shuvayanb/files_folders/main/nose_cone_images.png)

![](https://raw.githubusercontent.com/shuvayanb/files_folders/main/nose_cone_cell.png)

Newtonian theory is a local inclination method, *i.e*., local deflection angle $\theta$. According to this method, the pressure coefficient $C_p$ acting on a body at high-speed regime ($M_\mathrm{\infty}$>>1) depends on $\theta$, as:

$$ C_p = C_{p, max} \ \mathrm{sin}^2 \theta $$

In the Fig. above, the 3D object of interest can be represented using surface mesh or polygons. In this case, open source mesh generator Gmsh was used to obtain the surface triangulation for an arbitrary body. Each cell (or triangle) has in this case, 3 faces (or edges) with face normals $\textbf{n}_f$. The cell outward normal is indicated by $\textbf{n}_c$. 

The angle that the freestream velocity $V_\mathbf{\infty}$ makes with the local tangent plane at the $i^{th}$ cell is the local deflection angle $\theta_i$. We will first start by reading the nodes and element data from the mesh file. 

In [ ]:
# Read number of nodes and elements from the unstructured grid data

lookup1 = '$Nodes'
lookup2 = '$EndNodes'
lookup3 = '$Elements'
lookup4 = '$EndElements'

file = 'body.txt'

with open(file) as myFile:
    for num, line in enumerate(myFile, 1):
        if lookup1 in line:
            nodes_start = num + 1
            Nodes = next(myFile)
        if lookup2 in line:
            nodes_end = num
        if lookup3 in line:
            ele_start = num + 1
            Elements = next(myFile)       
        if lookup4 in line:
            ele_end = num
            

with open(file) as f_input:
    ndata = np.loadtxt(itertools.islice(f_input, nodes_start, nodes_end), delimiter=' ', skiprows=0)

nodes = ndata[:,1:4]
#print(nodes.shape)
    
ele_col = []
counter = 0

with open(file) as myFile:
    for num, line in enumerate(myFile, 1):
        if num>ele_start+1 and num<=ele_end+1:
            var = line.split(" ")
            if int(var[1])!=15 and int(var[1])!=1: # 15 is element type with 1 node point and 1 is element type with 2node line
                ele_col.append([int(var[5]),int(var[6]),int(var[7])])
                counter+= 1


elements = np.reshape(ele_col,(counter, 3))


<img src="https://raw.githubusercontent.com/shuvayanb/files_folders/main/cross_product.png" width="200"/>

Once the mesh file is loaded, we will start reading the geometric information in terms of node (or triangle verticex) coordinates, node indices forming a triangle. The second part is also crucial given that we will use this information to perform cell area-weighted averaging to obtain the corresponding interpolated pressure value at the nodes from the cells. 

From the image above, for any arbitrary element (or triangle) we calculate the face normals $\mathbf{n_1}$, $\mathbf{n_2}$ and $\mathbf{n_3}$ by taking the cross-product of any two of the edge vectors $\mathbf{e_1}$, $\mathbf{e_2}$. We then compute the unit normal vector at a face $i$ as 

$$\mathbf{n_{c, i}} =  \frac{\mathbf{n_i}}{\sqrt{n_1^2 + n_2^2 + n_3^2}}$$

In [ ]:
node_x = []
node_y = []
node_z = []

for i in range(len(nodes)):

    node_x.append(nodes[i,0])
    node_y.append(nodes[i,1])
    node_z.append(nodes[i,2])
    

vertex1 = np.zeros((len(elements),3))
vertex2 = np.zeros((len(elements),3))
vertex3 = np.zeros((len(elements),3))

for i in range(len(elements)):

    vertex1[i] = node_x[elements[i,0]-1],node_y[elements[i,0]-1],node_z[elements[i,0]-1] # triangle vertices
    vertex2[i] = node_x[elements[i,1]-1],node_y[elements[i,1]-1],node_z[elements[i,1]-1] # (-1) as the present indexing starts at 2
    vertex3[i] = node_x[elements[i,2]-1],node_y[elements[i,2]-1],node_z[elements[i,2]-1]
    
cell_cent = (vertex1+vertex2+vertex3)/3.0 # cell centroids

# arbitrary point

q = np.asarray(nodes.flatten())  

x_arb = 0.0
y_arb = 0.0
z_arb = 0.0

for i in range(0,int(Nodes),3):
    x_arb = x_arb + q[i];
    y_arb = y_arb + q[i+1];
    z_arb = z_arb + q[i+2];

x_arb = x_arb/len(nodes)
y_arb = x_arb/len(nodes)
z_arb = x_arb/len(nodes)


# edge vector for cross product

cell_edge_e0 = vertex2-vertex1
cell_edge_e1 = vertex3-vertex1

face_nor = 0.5*np.cross(cell_edge_e0,cell_edge_e1) # face normals from cross-product

nf1 = []
nf2 = []
nf3 = []

# dot product with arb vector

for i in range(len(elements)):
 
    nf1.append(face_nor[i,0])
    nf2.append(face_nor[i,1])
    nf3.append(face_nor[i,2])

    vecx = (x_arb - cell_cent[i,0]);
    vecy = (y_arb - cell_cent[i,1]);
    vecz = (z_arb - cell_cent[i,2]);
    
    temp = vecx*nf1[i] + vecy*nf2[i] + vecz*nf3[i]
    
    if temp<0: # to ensure all normals point either outward or inwards
        nf1[i] = -1* nf1[i]
        nf2[i] = -1* nf2[i]
        nf3[i] = -1* nf3[i]
        
    face_nor[i] = nf1[i], nf2[i], nf3[i]
    
face_ar = np.sqrt(np.sum(face_nor**2,axis=1))        # area of each triangle

nc = face_nor/face_ar[:,np.newaxis]        # Face unit outward normal


We will make some assumptions about the incoming flow while sticking to the high-speed flow regime, specifially hypersonic. 

The local inclination angle $\theta$ is obtained as:

$$ \theta = cos^{-1} \frac{\mathbf{n_c V}}{|n_c V|} $$

According to the modified Newtonian theory, the pressure acting on a body at high-speed regime can be computed as:

$$ C_p = C_{p, max} \ \mathrm{sin}^2 \theta $$

where, $C_{p, max}$ is evaluated as,

$$ C_{p, max} = \frac{2}{\gamma M_{\infty}^2} \Big \{ \Big [ \frac{(\gamma+1)^2 M_{\infty}^2}{4\gamma M_{\infty}^2 - 2(\gamma-1)} \Big]^{\frac{\gamma}{\gamma-1}} \Big [ \frac{ 1-\gamma + 2 \gamma M_{\infty}^2}{\gamma + 1} \Big ] - 1 \Big \} $$

where, $M_{\infty}$ is the freestream Mach number and $\gamma$ is the ratio of specific heats. The static pressure is related to pressure coefficient $C_p$ as:

$$C_p = \frac{p - p_{\infty}}{\frac{1}{2} \gamma p_{\infty} M_{\infty}^2 }$$

In [ ]:
gamma = 1.4           # ratio of specific heats
p_inf = 100           # freestream static pressure
M_inf = 7.0           # freestream Mach number
R = 287.0             # Gas constant
T_inf = 70.0          # freestream static temperature
cross_area = 0.00383  # base area

V_inf = M_inf*np.sqrt(gamma*R*T_inf);
vx = V_inf;
vy = 0.0
vz = 0.0

theta = np.zeros((len(elements),1))

for i in range(len(elements)):

    dot = ((vx*nc[i,0]) + (vy*nc[i,1]) + (vz*nc[i,2]));
    mod = np.sqrt(pow(vx,2) + pow(vy,2) + pow(vz,2)) * np.sqrt(pow(nc[i,0],2) + pow(nc[i,1],2) + pow(nc[i,2],2))
    
    theta[i] = (math.acos(dot/mod)*(180.0/np.pi))
    
constant1 = (pow((pow(gamma+1,2)*pow(M_inf,2))/((4*gamma*pow(M_inf,2))-(2*gamma-2)),(gamma/(gamma-1))) * ((1-gamma+(2*gamma*pow(M_inf,2)))/(gamma+1)));
constant2 = (gamma/2)*pow(M_inf,2);
CPmax = (constant1-1)/constant2


In [ ]:
drag = 0.0
lift = 0.0
cross_force = 0.0

fx = np.zeros((len(elements),1))
fy = np.zeros((len(elements),1))
fz = np.zeros((len(elements),1))
cp = np.zeros((len(elements),1))
pr = np.zeros((len(elements),1))

for i in range(len(elements)):
    
    if theta[i]>90.0:

        cp[i] = 0;

        pr[i] = 0
        
    if theta[i]<90.0:
        
        phi = 90 - theta[i];
        rad = ((phi)*(np.pi/180.0));
        
        cp[i] = CPmax*pow(np.sin(rad),2);

        pr[i] = (((0.5)*(gamma)*(p_inf)*(pow(M_inf,2)))*(cp[i]) + (p_inf));
                

    fx[i] = (pr[i]*face_ar[i]*nc[i,0]);
    fy[i] = (pr[i]*face_ar[i]*nc[i,1]);
    fz[i] = (pr[i]*face_ar[i]*nc[i,2]);

    drag = drag + fx[i];
    lift = lift + fy[i];
    cross_force = cross_force + fz[i]

        
cd = abs(drag/(0.5*pow(M_inf,2)*gamma*p_inf*cross_area))
cl = abs(lift/(0.5*pow(M_inf,2)*gamma*p_inf*cross_area))
cs = abs(cross_force/(0.5*pow(M_inf,2)*gamma*p_inf*cross_area))

The following cell interpolates the cell-ceontoid values to the nodes using area-weighted averaging.

In [ ]:
node_p = np.zeros((len(nodes),1))
node_denom = np.zeros((len(nodes),1)) 


for i in range(0,len(elements),1):
    for j in range(0,3,1): # 3 stands for number of vertices representing a triangle
        
        node_p[elements[i,j]-1] = node_p[elements[i,j]-1] +  (pr[i] * face_ar[i])
        
        node_denom[elements[i,j]-1] = node_denom[elements[i,j]-1] + face_ar[i]

        
for i in range(0,len(nodes),1):
    
    if node_denom[i]==0:
        node_denom[i] = 1e-6
    
    node_p[i] = node_p[i]/node_denom[i]



In [ ]:
with open('flowfield_paraview.dat','w')as f:
    f.writelines('VARIABLES="X","Y","Z","PR"')
    f.writelines('\n')
    f.writelines('ZONE N=%d E=%d F=FEPOINT ET=QUADRILATERAL' %(int(Nodes), counter))
    f.writelines('\n')
    
    for i in range(1,int(Nodes)+1,1):
        f.writelines('%f %f %f %f\n' %(nodes[i-1,0],nodes[i-1,1],nodes[i-1,2],node_p[i-1]))

    for i in range(0,counter,1):
        f.writelines('%d %d %d %d\n' %(elements[i-1,0],elements[i-1,1],elements[i-1,2],elements[i-1,2]))

The above cell will generate the output file containing the pressure flowfield. This can be easibly viewed using open source visualisation tool Paraview. 

For more information about how to use such local inclination methods for shape optimisation of rocket nose cone  or aeroshells, feel free to read this manuscript:

1. Brahmachary, S., Natarajan, G., and Sahoo, N.,"On Maximum Ballistic Coefficient Axisymmetric Geometries in Hypersonic Flows". *Journal of Spacecrafts and Rockets*, 2017

2. Theisinger, J.E., Braun, R.D., " Multiobjective Hypersonic Entry Aeroshell Shape Optimization", *AIAA 2008-5873*, 2008


Part of this code was initially written by Rohitasva Singh Jhala in C++, who worked as a research intern at Indian Institute of Technology Guwahati under prof. Vinayak Kulkarni. This code since then has been re-written in Python with other changes such as evaluation of face normals, cell-node interpolation, etc. 